In [1]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from matplotlib import cm

2021-10-15 10:51:04.920432: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
%matplotlib qt5

In [3]:
def plotImg(I, _3d = True, vmax=1):
    if _3d:
        fig, ax = plt.subplots(subplot_kw={"projection": "3d"})    

        X, Y = np.meshgrid(np.linspace(-1.0,1.0,I.shape[0]), np.linspace(-1.0,1.0, I.shape[1]))
        ax.plot_surface(X, Y, I, cmap=cm.coolwarm, linewidth=0, antialiased=False) 
    else:
        plt.imshow(I, vmin=0, vmax=vmax)
        
    plt.pause(0.0001)

In [4]:
def model(X, Y, O, framework, expo):
    ux, uy, s2, p = O
    
    z1 = framework.exp( -( ( ((X-ux)**2.0)/(2.0*s2) + ((Y-uy)**2.0)/(2.0*s2) ) )**expo )
    
    return p*z1




def fitImg(img):

    expo = 4.0
    
    t = np.linspace(0,2*np.pi,32)
    
    X, Y = tf.meshgrid(np.linspace(-1.0,1.0,img.shape[0]), np.linspace(-1.0,1.0, img.shape[1]))


    y = img


    max_epoch = 1000
    lossFnc = tf.keras.losses.mean_squared_error
    optimizer = tf.keras.optimizers.SGD(learning_rate = 2.0)
    _VARIABLE = type(tf.Variable(0))

    ux  = tf.Variable(0.0, dtype=tf.float64)
    uy  = tf.Variable(0.0, dtype=tf.float64)
    s2  = tf.Variable(0.5, dtype=tf.float64)
    p   = tf.Variable(0.1, dtype=tf.float64)

    O         = [ux,   uy,   s2,  p]
    trainable = [True, True, True, True]

    # eta = 1.0
    for epoch in range(max_epoch):

        with tf.GradientTape(persistent=True) as tape:

            variables = [x[0] for x in zip(O, trainable) if x[1]] 

            tape.watch(variables)

            yHat = model(X, Y, O, tf, expo)

            loss = tf.reduce_mean(lossFnc(y, yHat))

        grads = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(grads, variables))

        if epoch%10 == 0:
            R = 1.4*(2**expo)*np.sqrt(s2)
    #         print([x.numpy() for x in O])
    #         print([x.numpy() for x in grads])
            #print('batch : %d of %d (loss = %.4f)'%(epoch, max_epoch, loss))   
            plt.subplot(1,2,1)
            plt.cla()
            plotImg( y , False, 0.1)
            plt.plot(15.5+uy+R*np.cos(t), 15.5+ux+R*np.sin(t), linewidth=3, color='blue')
            plt.subplot(1,2,2)
            plt.cla()
            plotImg( yHat , False, 0.1)  
            plt.plot(15.5+ux+R*np.cos(t), 15.5+uy+R*np.sin(t), linewidth=3, color='red')
            plt.pause(0.0001)
            
            
    return O[2].numpy()

In [9]:
pwd

'/home/vieira/Downloads'

In [5]:
#%% Read Image data
for ind in range(19):

    # Read image
    imfilename = '/home/vieira/particle-density/results/10-microns particles-60X/crop_centered/crop_centered_' + str(ind) +'.tif'

    I = cv2.imread(imfilename, cv2.IMREAD_GRAYSCALE)

    #%% FIT GAUSSIAN
    new_shape = (32, 32)
    img = cv2.resize(I, new_shape, interpolation = cv2.INTER_AREA)
    
    bg = (np.mean(img[:4,:4]) + np.mean(img[:4,-4:]) + np.mean(img[-4:,:4]) + np.mean(img[-4:,-4:]))/4.0
    img = (img - bg)/255.0    
    
    s2 = fitImg(img)
    
    print(ind, s2)

error: OpenCV(4.5.1) ../modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


NameError: name 's21' is not defined

In [55]:
D = np.array([ [0, 0.03663217360424665],
[1, 0.036787580602119924],
[2, 0.03715351708033477],
[3, 0.03623278369863544],
[4, 0.0371574409780778],
[5, 0.03763849356936157],
[6, 0.03672058368173269],
[7, 0.04312988439675552],
[8, 0.05086492335023436],
[9, 0.07697191289543139],
[10, 0.08584123905352231],
[11, 0.1142007230167168],
[12, 0.13416394695657063],
[13, 0.16830583704790883],
[14, 0.21115452730389225],
[15, 0.2802665068904719],
[16, 0.32555063469517054],
[17, 0.40193664271089263],
[18, 0.43246089183084824] ])

In [58]:
plt.plot(D[:,0], D[:,1])